# **What is Ad-stock and why do we need it ?**
Ad-stock, or goodwill, is the cumulative value of a brand’s advertising at a given point in time. For example, if any company is advertising its product over 10 weeks, then for any given week t spending value would be **X + Past Week Fractional Amount**.
Ad-stock theory states that advertising is not immediate and has diminishing returns, meaning that its influential power decreases over time, even if more money is allocated to it. Therefore, time regression analysis will help marketers to understand the potential timeline for advertising effectiveness and how to optimise the marketing mix to compensate for these factors

Please follow this PDF for more understanding on mathematical formula - https://github.com/yug95/MachineLearning/blob/master/Market%20mix%20model/MMX%20Method.pdf

In [ ]:
import pandas as pd
import numpy as np
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.api import OLS
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('../input/sample-media-spends-data/Sample Media Spend Data.csv')

In [ ]:
media_columns = ['Overall_Views', 
                 'Google_Impressions', 
                 'Email_Impressions',
                 'Facebook_Impressions', 
                 'Affiliate_Impressions',
                 'Paid_Views', 
                 'Organic_Views' 
                ]

## Adstock calculation at Week level Granularity

In this secction, Iterating for all the media channel and then setting up decay rate & Saturation value, ***Decay rate would vary from 0.1 to 1 in interval of 0.1*** Lag defines upto what week you want to consider in 1 go. suppose you are at week 50 and want to calculate ad-stock value for week 51 then upto what week number from week- 50 to 1 you want factors to include. If we consider lag value =2 then at week 51 calculation, it will consider only week 49 & 48 factor.

In [ ]:
combined_df = pd.DataFrame()
for view in media_columns:
    print(view)
    temp = pd.DataFrame()
    for alpha in np.arange(0.1,1,0.2):
        Adstock_factor = np.round(alpha,1)
        for lag in [113]:
            for row in range(len(data)):
                x = data.loc[row,view]
                for j in range(1,lag+1):
                    if j< row:
                        x = x + (Adstock_factor ** j) * data.loc[row-j,view]
                        x = x
                    else:
                        x = x
                temp.loc[row,view+str(Adstock_factor)+str("_")+str("_113")] = x

    combined_df = pd.concat([combined_df,temp],axis=1)

In [ ]:
combined_df

Merging Data frame with Original Data columns

In [ ]:
combined_df = pd.concat([data,combined_df],axis=1)

In [ ]:
combined_df

Plot the difference between Raw Spend vs Ad-stock Spend for any media channel at any Division

In [ ]:
A_Div_Data = combined_df[combined_df.Division=='A']
plt.figure(figsize=(15,8))
plt.plot(A_Div_Data.Calendar_Week,A_Div_Data.Google_Impressions,'k',label='Raw Spend')
sns.barplot(A_Div_Data.Calendar_Week,A_Div_Data['Google_Impressions0.5__113'],label='Ad-stock Spend')
plt.xticks(rotation=90)
plt.legend()
plt.show()

In [ ]:
combined_df.to_excel('Adstock Combination with beta at overall level.xlsx')

## Adstock calculation at Division/State week granularity

Same as above, here we just included additional division level granularity. Hence it will treat each division spend as separate entity

In [ ]:
Division_State_week_level = data.copy()

In [ ]:
state_level_adstock_df = pd.DataFrame()

for division in Division_State_week_level.Division.unique():
    temp = pd.DataFrame()
    division_data = Division_State_week_level[Division_State_week_level.Division == division]
    division_data.reset_index(inplace=True)
    for view in media_columns:
        for alpha in np.arange(0.1,1,0.3):
            Adstock_factor = np.round(alpha,1)
            for lag in [113]:
                for row in range(len(division_data)):
                    x = division_data.loc[row,view]
                    for j in range(1,lag+1):
                        if j<= row:
                            x = x + (Adstock_factor ** j) * division_data.loc[row-j,view]
                        else:
                            x = x
                    temp.loc[row,view+str(Adstock_factor)+str("_")+str(lag)] = x              
    state_level_adstock_df = pd.concat([state_level_adstock_df,temp],axis=0,ignore_index=True)

In [ ]:
state_level_adstock_df = pd.concat([data,state_level_adstock_df],axis=1)

In [ ]:
state_level_adstock_df

In [ ]:
N_Div_Data = state_level_adstock_df[state_level_adstock_df.Division=='N']
plt.figure(figsize=(15,8))
plt.plot(N_Div_Data.Calendar_Week,N_Div_Data.Facebook_Impressions,'k',label='Raw Spend')
sns.barplot(N_Div_Data.Calendar_Week,N_Div_Data['Facebook_Impressions0.7_113'],label='Ad-stock Spend')
plt.xticks(rotation=90)
plt.legend()
plt.show()

In [ ]:
state_level_adstock_df.to_excel('Adstock_Combination with State_Division_granularity.xlsx')